In [1]:
"""
Example script for defining and using FeatureGenerators in AutoGluon Tabular.
FeatureGenerators act to clean and prepare the data to maximize predictive accuracy in downstream models.
FeatureGenerators are stateful data preprocessors which take input data (pandas DataFrame) and output transformed data (pandas DataFrame).
FeatureGenerators are first fit on training data through the .fit_transform() function, and then transform new data through the .transform() function.
These generators can do anything from filling NaN values (FillNaFeatureGenerator), dropping duplicate features (DropDuplicatesFeatureGenerator), generating ngram features from text (TextNgramFeatureGenerator), and much more.
In AutoGluon's TabularPredictor, the input data is transformed via a FeatureGenerator before entering a machine learning model. Some models use this transformed input directly and others perform further transformations before making predictions.

This example is intended for advanced users that have a strong understanding of feature engineering and data preparation.
Most users can get strong performance without specifying custom feature generators due to the generic and powerful default feature generator used by AutoGluon.
An advanced user may wish to create a custom feature generator to:
    1. Experiment with different preprocessing pipelines to improve model quality.
    2. Have full control over what data is being sent to downstream models.
    3. Migrate existing pipelines into AutoGluon for ease of use and deployment.
    4. Contribute new feature generators to AutoGluon.
"""

################
# Loading Data #
################

import pandas as pd
import os
import torch
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np



/user/mahaohui/miniconda3/envs/autoML/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from autogluon.core.utils.loaders import load_pd
train_data  = load_pd.load('./train.csv')
test_data  = load_pd.load('./test.csv')
train_data.head()
label = 'solubility'
splitted_valid_data = train_data[train_data["fold"] ==0.0]
splitted_train_data = train_data[train_data["fold"] !=0.0]
# valid_data = valid_data[["seq","solubility"]]
# train_data = train_data[["seq","solubility"]]
# # train_data = train_data[:1000]
# X = train_data[["seq"]]
# y = train_data[label]
# X_test = test_data[["seq","solubility"]]
# y_test = test_data[label]


In [21]:
print(len(train_data))
print(len(test_data))

11224
1323


In [6]:
concatenated_df = pd.concat([train_data,test_data], axis=0)

In [22]:
# concatenated_df  = concatenated_df.fillna(value=-1)
print(concatenated_df)

               sid  solubility  \
0     AaCD00331182           1   
1     AaCD00331183           1   
2     AaCD00331184           1   
3     AaCD00331185           1   
4     AaCD00331621           1   
...            ...         ...   
1318          ZR72           1   
1319          ZR74           1   
1320          ZR75           1   
1321          ZR78           1   
1322          ZR93           1   

                                                    seq  fold  
0     MTYKDGTYSSDGTYTSPNGLETVGVELTLAADKVSAVNITVHPSNP...   0.0  
1     MTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPADRKKT...   1.0  
2     MKAEGNTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPA...   1.0  
3     MQSFNSGSTKIHNAFPEESTRPQKAEGNTAMNILVLGSDSRGSSDA...   1.0  
4     MNAPVKFEYFKNPKNRELTAVELEAFAKELDQIKQEVLDDIGEKDA...   2.0  
...                                                 ...   ...  
1318  MGGYKGIKADGGKVNQAKQLAAKIAKDIEACQKQTQQLAEYIEGSD...   NaN  
1319  MAFTLSAIQQAHQQFTGVDFPKLFKAFKDMGMTYNIVNIQDGTATY...   NaN  
1320  MASKYGIND

In [23]:
concatenated_df.dtypes

sid            object
solubility      int64
seq            object
fold          float64
dtype: object

In [24]:
concatenated_df.head()

,sid,solubility,seq,fold
0,AaCD00331182,1,MTYKDGTYSSDGTYTSPNGLETVGVELTLAADKVSAVNITVHPSNP...,0.0
1,AaCD00331183,1,MTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPADRKKT...,1.0
2,AaCD00331184,1,MKAEGNTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPA...,1.0
3,AaCD00331185,1,MQSFNSGSTKIHNAFPEESTRPQKAEGNTAMNILVLGSDSRGSSDA...,1.0
4,AaCD00331621,1,MNAPVKFEYFKNPKNRELTAVELEAFAKELDQIKQEVLDDIGEKDA...,2.0


In [25]:
#####################################
# Create a custom feature generator #
#####################################

from pandas import DataFrame
import torch
from autogluon.features.generators import AbstractFeatureGenerator
from autogluon.common.features.types import R_INT,R_FLOAT,R_OBJECT,R_CATEGORY,S_TEXT_AS_CATEGORY 
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import pad_sequence

# Feature generator to add k to all values of integer features.
class PlusKFeatureGenerator(AbstractFeatureGenerator):
    def __init__(self, k, **kwargs):
        super().__init__(**kwargs)
        self.k = k

    def _fit_transform(self, X: DataFrame, **kwargs) -> (DataFrame, dict):
        # Here we can specify any logic we want to make a stateful feature generator based on the data.
        # Just call _transform since this isn't a stateful feature generator.
        X_out = self._transform(X)
        # return the output and the new special types of the data. For this generator, we don't add any new special types, so just return the input special types
        return X_out, self.feature_metadata_in.type_group_map_special

    def _transform(self, X: DataFrame) -> DataFrame:
        # Here we can specify the logic taken to convert input data to output data post-fit. Here we can reference any variables created during fit if the generator is stateful.
        # Because this feature generator is not stateful, we simply add k to all features.
        return X + self.k

    @staticmethod
    def get_default_infer_features_in_args() -> dict:
        default_infer_features = dict(valid_raw_types=[R_CATEGORY]) 
        print(default_infer_features)
        return default_infer_features  # This limits input features to only integers. We can assume that the input to _fit_transform and _transform only contain the data post-applying this filter.


    
class net_charge_Generator(AbstractFeatureGenerator):
    def __init__(self,  **kwargs):
        super().__init__(**kwargs)

    def _fit_transform(self, X: DataFrame, **kwargs) -> (DataFrame, dict):
        # Here we can specify any logic we want to make a stateful feature generator based on the data.
        # Just call _transform since this isn't a stateful feature generator.
        X_out = self._transform(X)
        # return the output and the new special types of the data. For this generator, we don't add any new special types, so just return the input special types
        return X_out, self.feature_metadata_in.type_group_map_special

    def _transform(self, X: DataFrame) -> DataFrame:
        # Here we can specify the logic taken to convert input data to output data post-fit. Here we can reference any variables created during fit if the generator is stateful.
        # Because this feature generator is not stateful, we simply add k to all features.

        def net_charge(seq):
            # Define the pKa values of the amino acids at pH 7.4
            pKa = {'D': 3.9, 'E': 4.3, 'H': 6.0, 'C': 8.3, 'Y': 10.1, 'K': 10.8, 'R': 12.5,
                   'A': 0, 'G': 0, 'I': 0, 'L': 0, 'M': 0, 'F': 0, 'P': 0, 'S': 0, 'T': 0,
                   'W': 0, 'V': 0}
            # Count the number of each type of amino acid in the sequence
            aa_count = {aa: seq.count(aa) for aa in pKa.keys()}

            # Calculate the net charge of the sequence using the pKa values
            net_charge = sum([-1 * aa_count[aa] * (10 ** (-pKa[aa])) for aa in ['D', 'E']]) \
                         + sum([aa_count[aa] * (10 ** (-pKa[aa])) for aa in ['K', 'R', 'H']]) \
                         + sum([aa_count[aa] for aa in ['C', 'Y', 'K', 'R']])
            return net_charge
    
        print("X:",X)
        df = pd.DataFrame(columns=['net_charge'])
        for column in X.columns:
            df['net_charge'] = X['seq'].apply(net_charge)
        
        return df

    @staticmethod
    def get_default_infer_features_in_args() -> dict:
        default_infer_features = dict(valid_raw_types=[R_OBJECT]) 
        print(default_infer_features)
        return default_infer_features  # This limits input features to only integers. We can assume that the input to _fit_transform and _transform only contain the data post-applying this filter.


class count_charge_Generator(AbstractFeatureGenerator):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _fit_transform(self, X: DataFrame, **kwargs) -> (DataFrame, dict):
        # Here we can specify any logic we want to make a stateful feature generator based on the data.
        # Just call _transform since this isn't a stateful feature generator.
        X_out = self._transform(X)
        # return the output and the new special types of the data. For this generator, we don't add any new special types, so just return the input special types
        return X_out, self.feature_metadata_in.type_group_map_special

    def _transform(self, X: DataFrame) -> DataFrame:
        # Here we can specify the logic taken to convert input data to output data post-fit. Here we can reference any variables created during fit if the generator is stateful.
        # Because this feature generator is not stateful, we simply add k to all features.

        def count_chargeed(seq):
            charged = ['D','E','K','R','H']
            polar = ['S','T','T','Q','C']
            aromatic = ['Y']

            hdrophobic = ['A','V','L','I','M','F','W']
            neutral = ['P','G']

            charged_counter = 0
            polar_counter = 0
            aromatic_counter = 0
            hdrophobic_counter = 0
            neutral_counter = 0

            for c in seq:
                if c in charged:
                    charged_counter+=1
                elif c in polar:
                    polar_counter+=1
                elif c in aromatic:
                    aromatic_counter+=1
                elif c in hdrophobic:
                    hdrophobic_counter+=1
                elif c in neutral:
                    neutral_counter+=1
            return (charged_counter,polar_counter,aromatic_counter,hdrophobic_counter,neutral_counter)

        df = pd.DataFrame(columns=['charged', 'polar', 'aromatic', 'hdrophobic', 'neutral'])
        for column in X.columns:
            df[['charged', 'polar', 'aromatic', 'hdrophobic', 'neutral']] = X[column].apply(count_chargeed).tolist()
        return df

    @staticmethod
    def get_default_infer_features_in_args() -> dict:
        default_infer_features = dict(valid_raw_types=[R_OBJECT]) 
        print(default_infer_features)
        return default_infer_features  # This limits input features to only integers. We can assume that the input to _fit_transform and _transform only contain the data post-applying this filter.

    
class one_hot_Generator(AbstractFeatureGenerator):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _fit_transform(self, X: DataFrame, **kwargs) -> (DataFrame, dict):
        # Here we can specify any logic we want to make a stateful feature generator based on the data.
        # Just call _transform since this isn't a stateful feature generator.
        X_out = self._transform(X)
        # return the output and the new special types of the data. For this generator, we don't add any new special types, so just return the input special types
        return X_out, self.feature_metadata_in.type_group_map_special

    def _transform(self, X: DataFrame) -> DataFrame:
        # Here we can specify the logic taken to convert input data to output data post-fit. Here we can reference any variables created during fit if the generator is stateful.
        # Because this feature generator is not stateful, we simply add k to all features.
        letter_to_int = {'C': 0, 'P': 1, 'R': 2, 'N': 3, 'F': 4, 'K': 5, 'A': 6, 'H': 7, 'Y': 8, 'V': 9, 'L': 10, 'D': 11, 'G': 12, 'E': 13, 'Q': 14, 'M': 15, 'T': 16, 'S': 17, 'I': 18, 'W': 19, 'X':20}

        def one_hot_encoding(sequence,letter_to_int):
            letter_sequence = [letter_to_int[letter] for letter in sequence]

            encoded_tensor  = torch.zeros((len(letter_to_int),len(sequence)), dtype=torch.int64)
            for i in range(len(letter_sequence)):
                encoded_tensor[letter_sequence[i],i] = 1
            return encoded_tensor
        
        # Convert the protein sequences to one-hot encoding
        
        one_hot_df = pd.DataFrame()

        # get the first column 
        column = X.iloc[:, 0]
        sequences = column.tolist()
        one_hot_seqs = []
        for seq in sequences:
            one_hot_seq = one_hot_encoding(seq,letter_to_int)
            one_hot_seqs.append(one_hot_seq.flatten().numpy())

        #print("one_hot_seqs size",one_hot_seqs.shape)
        # Create a dataframe with separate columns for each amino acid position
        max_length = max(len(seq) for seq in sequences)
        column_name = [f'aa{i}_{aa}' for aa in letter_to_int for i in range(1, max_length+1) ]


        #print("one_hot_seqs shape",one_hot_seqs.shape)
        df = pd.DataFrame(one_hot_seqs,columns = column_name)
        df  = df.fillna(value=0).astype("bool")
        return df

    @staticmethod
    def get_default_infer_features_in_args() -> dict:
        default_infer_features = dict(valid_raw_types=[R_OBJECT]) 
        print(default_infer_features)
        return default_infer_features  # This limits input features to only integers. We can assume that the input to _fit_transform and _transform only contain the data post-applying this filter.


In [11]:
# one_hot_train = one_hot_Generator(verbosity=3,features_in=['seq'])
# one_hot_train_data = one_hot_train.fit_transform(X=train_data)
# one_hot_train_data  = one_hot_train_data.fillna(value=0, downcast='infer')
# print(one_hot_train_data.head(5))

In [26]:
from autogluon.features.generators import CategoryFeatureGenerator, AsTypeFeatureGenerator, BulkFeatureGenerator, DropUniqueFeatureGenerator, FillNaFeatureGenerator, PipelineFeatureGenerator, OneHotEncoderFeatureGenerator,IdentityFeatureGenerator
import copy
train_feature_generator = PipelineFeatureGenerator(
    generators=[
        # Stage 1: Convert feature types to be the same as during fit. Does not need to be specified.
        # Stage 2: Fill NaN values of data. Does not need to be specified.
        [  # Stage 3: Add 5 to all int features and convert all object features to category features. Concatenate the outputs of each.
            count_charge_Generator(),
            net_charge_Generator(),
            one_hot_Generator(verbosity=3,features_in=['seq']),
            #OneHotEncoderFeatureGenerator(),
            #CategoryFeatureGenerator(),
            IdentityFeatureGenerator(infer_features_in_args=dict(
                valid_raw_types=[R_INT, R_FLOAT])),
        ],
        # Stage 4: Drop any features which are always the same value (useless). Does not need to be specified.
     ],
    verbosity=3
)


one_hot_all_data = train_feature_generator.fit_transform(X=concatenated_df)
print(one_hot_all_data)

{'valid_raw_types': ['object']}
{'valid_raw_types': ['object']}
{'valid_raw_types': ['object']}
X:                 sid                                                seq
0      AaCD00331182  MTYKDGTYSSDGTYTSPNGLETVGVELTLAADKVSAVNITVHPSNP...
1      AaCD00331183  MTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPADRKKT...
2      AaCD00331184  MKAEGNTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPA...
3      AaCD00331185  MQSFNSGSTKIHNAFPEESTRPQKAEGNTAMNILVLGSDSRGSSDA...
4      AaCD00331621  MNAPVKFEYFKNPKNRELTAVELEAFAKELDQIKQEVLDDIGEKDA...
...             ...                                                ...
12542          ZR72  MGGYKGIKADGGKVNQAKQLAAKIAKDIEACQKQTQQLAEYIEGSD...
12543          ZR74  MAFTLSAIQQAHQQFTGVDFPKLFKAFKDMGMTYNIVNIQDGTATY...
12544          ZR75  MASKYGINDIVEMKKQHACGTNRFKIIRMGADIRIKCENCQRSIMI...
12545          ZR78  MNMHILYNLRTKHNLEIDELAQQLNEKYGTKYEAHQIWEWENHHHE...
12546          ZR93  IYYRGAHYMKVTDVRLRKIQTDGRMKALVSITLDEAFVIHDLRVIE...

[12547 rows x 2 columns]
      charged  polar  a

In [27]:
print(one_hot_all_data.dtypes)

charged         int64
polar           int64
aromatic        int64
hdrophobic      int64
neutral         int64
               ...   
aa749_W          bool
aa776_W          bool
aa810_W          bool
solubility       int8
fold          float64
Length: 13764, dtype: object


In [28]:
one_hot_train_data = one_hot_all_data[:len(train_data)]
one_hot_test_data = one_hot_all_data[len(train_data):]
print(one_hot_train_data)
print(one_hot_test_data)

       charged  polar  aromatic  hdrophobic  neutral  net_charge  aa2_C  \
0           20     24         3          32       13   10.999071  False   
1           62     51         6         120       31   30.996253  False   
2           64     51         6         121       32   31.996203  False   
3           69     59         6         125       35   33.996104  False   
4           92     52        18         137       46   70.997347  False   
...        ...    ...       ...         ...      ...         ...    ...   
11219       13     12         3          26       13    7.999424  False   
11220       13     12         3          29       14    7.999424  False   
11221       15     12         3          31       15    7.999172  False   
11222       32     19         2          47        9   16.998998  False   
11223       49     31         5          88       26   27.997996  False   

       aa3_C  aa4_C  aa5_C  ...  aa576_W  aa627_W  aa635_W  aa739_W  aa745_W  \
0      False  False

In [29]:
one_hot_valid_data1 = one_hot_train_data[one_hot_train_data["fold"] ==0.0]
one_hot_train_data1 = one_hot_train_data[one_hot_train_data["fold"] !=0.0]

In [30]:
one_hot_train_data1 = one_hot_train_data1.drop(["fold"],axis=1)
# one_hot_train_data1.astype(bool)                                            

In [31]:
one_hot_valid_data1 = one_hot_valid_data1.drop(["fold"],axis=1)
# one_hot_valid_data1.astype(bool)

In [32]:
print(one_hot_train_data1.dtypes)
print(one_hot_valid_data1.dtypes)

charged       int64
polar         int64
aromatic      int64
hdrophobic    int64
neutral       int64
              ...  
aa745_W        bool
aa749_W        bool
aa776_W        bool
aa810_W        bool
solubility     int8
Length: 13763, dtype: object
charged       int64
polar         int64
aromatic      int64
hdrophobic    int64
neutral       int64
              ...  
aa745_W        bool
aa749_W        bool
aa776_W        bool
aa810_W        bool
solubility     int8
Length: 13763, dtype: object


In [33]:
# DO THE ENCODING BEFOTRE SPLITING TRAIN VALID!!!!!!


In [34]:
predictor = TabularPredictor(label='solubility',eval_metric="precision").fit(train_data=one_hot_train_data1, tuning_data=one_hot_valid_data1, feature_generator=None)

No path specified. Models will be saved in: "AutogluonModels/ag-20230620_034218/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230620_034218/"
AutoGluon Version:  0.7.0
Python Version:     3.9.0
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 10 13:32:12 UTC 2021
Train Data Rows:    8281
Train Data Columns: 13762
Tuning Data Rows:    2943
Tuning Data Columns: 13762
Label Column: solubility
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['b

[1000]	valid_set's binary_logloss: 0.63024	valid_set's precision: 0.716032


	0.7195	 = Validation score   (precision)
	35.52s	 = Training   runtime
	2.04s	 = Validation runtime
Fitting model: LightGBM ...
	0.7238	 = Validation score   (precision)
	28.99s	 = Training   runtime
	2.04s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7012	 = Validation score   (precision)
	18.53s	 = Training   runtime
	0.26s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7012	 = Validation score   (precision)
	17.84s	 = Training   runtime
	0.29s	 = Validation runtime
Fitting model: CatBoost ...
	0.7197	 = Validation score   (precision)
	299.1s	 = Training   runtime
	12.19s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7048	 = Validation score   (precision)
	17.65s	 = Training   runtime
	0.3s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7012	 = Validation score   (precision)
	17.6s	 = Training   runtime
	0.26s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
/user/mahaohui/miniconda3/envs/autoML/lib/python3.9/site-packag

[1000]	valid_set's binary_logloss: 0.773446	valid_set's precision: 0.718647


	0.7201	 = Validation score   (precision)
	60.65s	 = Training   runtime
	2.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.7524	 = Validation score   (precision)
	2.43s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 2469.18s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230620_034218/")


In [37]:
predictor.evaluate(one_hot_test_data, silent=True)

{'precision': 0.6559766763848397,
 'accuracy': 0.6061980347694633,
 'balanced_accuracy': 0.5328479859358719,
 'mcc': 0.07601266744204434,
 'roc_auc': 0.5811971298907166,
 'f1': 0.7215392838054517,
 'recall': 0.8016627078384798}

In [38]:
predictor.evaluate(one_hot_valid_data1, silent=True)

{'precision': 0.7523510971786834,
 'accuracy': 0.7366632687733605,
 'balanced_accuracy': 0.6062508262459789,
 'mcc': 0.28663191681352157,
 'roc_auc': 0.6906642466839994,
 'f1': 0.8320693391115928,
 'recall': 0.9306834706737761}